# Data Augmentation

Après avoir nettoyé les données, nous pouvons maintenant procéder à l'augmentation des données. L'augmentation des données consiste à créer de nouvelles données à partir de données existantes en appliquant des transformations qui n'affecte pas les caractéristique pertinentes de l'audio.

Cela peut inclure des changements de tonalité, de vitesse, d'ajout de bruit, ect.

## Imports

In [2]:
import IPython.display as ipd
from scipy.signal import hilbert
import numpy as np
import pandas as pd
import scipy
from scipy.stats import skew
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import soundfile as sf

import tools

# change style
plt.style.use('ggplot')
import librosa
import librosa.display

from tqdm import tqdm
from datetime import datetime

tqdm.pandas()
import os
from glob import glob
import random
from typing import List, Tuple, Dict

from concurrent.futures import ThreadPoolExecutor, as_completed
import threading
import librosa

In [3]:
from params import SOUNDS_DATASET_PATH, SAMPLE_RATE, CLASS_COLORS, AIRS_PATH
from tools import play_audio, load_audio_file, pad_signal, get_all_audios_files

## Load Dataset

In [4]:
dataset_csv_path = os.path.join(SOUNDS_DATASET_PATH, 'dataset_features_selected_20230425.csv')
dataset = pd.read_csv(dataset_csv_path)

# set index
dataset.set_index('file_path', inplace=True)

print(f"Dataset shape: {dataset.shape}")
dataset

Dataset shape: (10300, 75)


,file_name,file_extension,class,split,duration,rms_log_mean,rms_log_max,rms_log_std,rms_log_diff_abs_mean,zcr_sum,...,mfcc_kurtosis_7,mfcc_kurtosis_11,spec_cent_std,spec_bw_mean,spec_bw_std,spec_flatness_mean,spec_flatness_std,spec_rolloff_std,spec_contrast_mean,spec_contrast_std
file_path,,,,,,,,,,,,,,,,,,,,,
G:\Shared drives\PFE - ING3 Mlamali\DrumClassifier - Sounds Dataset\Kick\Almighty Kick [CraddyMusic].wav,Almighty Kick [CraddyMusic],.wav,Kick,test,1.718753,-5.474137,-0.315035,1.542840,0.039382,0.074219,...,9.888011,9.223362,248.089962,66.760497,446.912802,0.885944,0.317821,20.798566,9.690083,2.764552
G:\Shared drives\PFE - ING3 Mlamali\DrumClassifier - Sounds Dataset\Kick\Basic Kick.wav,Basic Kick,.wav,Kick,train,0.466122,-3.507895,-0.203399,1.873567,0.163677,19.022461,...,1.460388,0.835710,5981.829061,6040.415139,2628.972936,0.120724,0.157800,1211.708854,13.097692,4.937484
G:\Shared drives\PFE - ING3 Mlamali\DrumClassifier - Sounds Dataset\Kick\Best (Kick).wav,Best (Kick),.wav,Kick,train,1.750000,-4.232851,-0.193866,1.077849,0.054044,84.785156,...,19.320816,12.730776,5104.549406,5234.423975,1497.614684,0.141212,0.075214,4439.359665,12.800571,4.841606
G:\Shared drives\PFE - ING3 Mlamali\DrumClassifier - Sounds Dataset\Kick\Demon_Kick.wav,Demon_Kick,.wav,Kick,train,8.000000,-5.234494,-0.581696,0.567349,0.015392,317.313477,...,72.417990,93.611100,1295.787569,5763.021065,620.968026,0.430654,0.058395,423.945173,14.293943,9.406386
G:\Shared drives\PFE - ING3 Mlamali\DrumClassifier - Sounds Dataset\Kick\Dope Kick.wav,Dope Kick,.wav,Kick,train,0.244762,-2.509092,-0.176139,1.624756,0.240633,0.715820,...,-1.250166,-0.503283,4437.548206,5035.442166,3342.453843,0.013908,0.032529,354.459640,13.789824,4.959522
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
G:\Shared drives\PFE - ING3 Mlamali\DrumClassifier - Sounds Dataset\Metal\Metal 0497.flac,Metal 0497,.flac,Metal,train,4.696168,-2.736829,-0.604934,0.561248,0.057590,19.468750,...,0.871423,7.330698,482.423116,4260.595436,1354.476792,0.001395,0.001316,557.852105,18.412576,8.553419
G:\Shared drives\PFE - ING3 Mlamali\DrumClassifier - Sounds Dataset\Metal\Metal 0498.flac,Metal 0498,.flac,Metal,train,5.926893,-2.607688,-0.517238,0.721487,0.046787,26.238281,...,-0.928046,-0.819818,1431.913235,3226.597229,1531.333314,0.006139,0.037385,442.183534,18.144347,5.442813
G:\Shared drives\PFE - ING3 Mlamali\DrumClassifier - Sounds Dataset\Metal\Metal 0499 H.flac,Metal 0499 H,.flac,Metal,train,1.532517,-3.548632,-0.833822,1.039219,0.043037,43.234375,...,-0.577570,2.168855,1286.190470,5448.071177,1077.816553,0.208723,0.213114,1278.056279,14.850453,7.317871


## Set Examples Audio

## Augmentation Pipeline

- Noise Addition : Add random noise to an audio signal
- Time Stretch : Stretch an audio signal in time
- Pitch Scaling : Scale the pitch of an audio signal
- Polarity Inversion : Invert the polarity of an audio signal
- Random Gain : Add random gain to an audio signal
- Time Shift : Shift an audio signal in time
- Impulse Response : Apply an impulse response to an audio signal
- Filtering (Low Pass, High Pass, Band Pass, Band Reject) : Filter an audio signal
- Reverb : Add reverberation to an audio signal
- Distortion : Add distortion to an audio signal
- Compression : Apply compression to an audio signal
- Chorus : Add chorus to an audio signal
- Flanger : Add flanger to an audio signal


In [5]:
from audiomentations import (
    Compose, AddGaussianNoise, TimeStretch, PitchShift, PolarityInversion,
    Gain, ApplyImpulseResponse, Shift, LowPassFilter, HighPassFilter,
    BandPassFilter, BandStopFilter
)

In [6]:
ex_path = "G:\Shared drives\PFE - ING3 Mlamali\DrumClassifier - Sounds Dataset\Snare\Snare 0201.flac"
y, sr = load_audio_file(ex_path)
play_audio(y=y, sr=sr, title='Original Signal')

> Original Signal


In [20]:
def augment_audio(y, sample_rate,
                  apply_noise=False, max_noise_amplitude=0.05,p_noise=1.0,
                  apply_time_stretch=False, min_stretch_rate=0.8, max_stretch_rate=1.2,p_time_stretch=1.0,
                  apply_pitch_scaling=False, min_semitones=-5, max_semitones=5, p_pitch_scaling=1.0,
                  apply_polarity_inversion=False,p_polarity_inversion=1.0,
                  apply_random_gain=False, min_gain_db=-12, max_gain_db=12, p_random_gain=1.0,
                  apply_impulse_response=False, ir_path=AIRS_PATH,p_impulse_response=1.0,
                  apply_time_shift=False,min_shift_fraction=-0.5, max_shift_fraction=0.5, fade_shift=True,
                  fade_shift_duration=0.01, p_time_shift=1.0,
                  apply_filtering=False, filter_type='low_pass', min_filter_freq=100, max_filter_freq=10000, p_filtering=1.0):
    """
    Augment an audio signal with the given parameters.

    Args:
        y (np.ndarray): Audio signal
        sample_rate (int): Sample rate of the audio signal
        apply_noise (bool): Whether to apply noise augmentation
        max_noise_amplitude (float): Maximum amplitude of the noise to add
        apply_time_stretch (bool): Whether to apply time stretch augmentation
        min_stretch_rate (float): Minimum stretch rate
        max_stretch_rate (float): Maximum stretch rate
        apply_pitch_scaling (bool): Whether to apply pitch scaling augmentation
        min_semitones (int): Minimum pitch scaling in semitones
        max_semitones (int): Maximum pitch scaling in semitones
        apply_polarity_inversion (bool): Whether to apply polarity inversion augmentation
        apply_random_gain (bool): Whether to apply random gain augmentation
        min_gain_db (float): Minimum gain in dB
        max_gain_db (float): Maximum gain in dB
        apply_impulse_response (bool): Whether to apply impulse response augmentation
        ir_path (str): Path to the impulse response file
        max_ir_gain_db (float): Maximum impulse response gain in dB
        apply_time_shift (bool): Whether to apply time shift augmentation
        max_shift_fraction (int): Maximum number of samples to shift
        apply_filtering (bool): Whether to apply filtering augmentation
        filter_type (str): Type of filter to apply.
        min_filter_freq (int): Minimum filter frequency
        max_filter_freq (int): Maximum filter frequency

    """
    # parameters strings with abbreviated names and values (with sub parameters)
    params_str = f"noise_{int(apply_noise)}__ts_{int(apply_time_stretch)}__ps_{int(apply_pitch_scaling)}__pi_{int(apply_polarity_inversion)}__rg_{int(apply_random_gain)}__ir_{int(apply_impulse_response)}__ts_{int(apply_time_shift)}__fl_{int(apply_filtering)}"
    augmentations = []

    if apply_noise:
        augmentations.append(AddGaussianNoise(max_amplitude=max_noise_amplitude, p=p_noise))
    if apply_time_stretch:
        augmentations.append(TimeStretch(min_rate=min_stretch_rate, max_rate=max_stretch_rate, p=p_time_stretch))
    if apply_pitch_scaling:
        augmentations.append(PitchShift(min_semitones=min_semitones, max_semitones=max_semitones, p=p_pitch_scaling))
    if apply_polarity_inversion:
        augmentations.append(PolarityInversion(p=p_polarity_inversion))
    if apply_random_gain:
        augmentations.append(Gain(min_gain_in_db=min_gain_db, max_gain_in_db=max_gain_db, p=p_random_gain))
    if apply_impulse_response and ir_path is not None:
        augmentations.append(ApplyImpulseResponse(ir_path=ir_path, p=p_impulse_response))
    if apply_time_shift:
        augmentations.append(Shift(min_fraction=min_shift_fraction, max_fraction=max_shift_fraction, fade=fade_shift,
                                   fade_duration=fade_shift_duration, p=p_time_shift))
    if apply_filtering:
        if filter_type == 'low_pass':
            augmentations.append(LowPassFilter(min_cutoff_freq=min_filter_freq, max_cutoff_freq=max_filter_freq, p=p_filtering))
        elif filter_type == 'high_pass':
            augmentations.append(HighPassFilter(min_cutoff_freq=min_filter_freq, max_cutoff_freq=max_filter_freq, p=p_filtering))
        elif filter_type == 'band_pass':
            augmentations.append(BandPassFilter(min_center_freq=min_filter_freq, max_center_freq=max_filter_freq, p=p_filtering))
        elif filter_type == 'band_reject':
            augmentations.append(BandStopFilter(min_center_freq=min_filter_freq, max_center_freq=max_filter_freq, p=p_filtering))

    augmenter = Compose(augmentations)
    return augmenter(y, sample_rate), params_str

### Noise Addition

In [26]:
from params import CLASS_COLORS

CLASS_COLORS.keys()

dict_keys(['Metal', 'Conga', 'Kick', 'Hat Open (Cymbal)', 'Ride (Cymbal)', '808', 'Snare', 'Bell', 'Hat Closed (Cymbal)', 'Crash (Cymbal)', 'Tom', 'Clap', 'Snap', 'Shaker', 'Triangle'])

thomas.danguilhen@estaca.eu
Pour chacune des augmentations possibles donne moi une valeur p (% de chance d'être appliqué). Je veux les valeur p les plus pertinentes, sachant que je compte augmenter ces class de drums : ['Metal', 'Conga', 'Kick', 'Hat Open (Cymbal)', 'Ride (Cymbal)', '808', 'Snare', 'Bell', 'Hat Closed (Cymbal)', 'Crash (Cymbal)', 'Tom', 'Clap', 'Snap', 'Shaker', 'Triangle']

In [25]:
play_audio(y=y, sr=sr, title='Original Signal')

for max_noise_amplitude in [0.001, 0.005, 0.01, 0.05]:
    augmented_y, params_str = augment_audio(y=y, sample_rate=sr, apply_noise=True,
                                            max_noise_amplitude=max_noise_amplitude)
    play_audio(y=augmented_y, sr=sr, title=f'Augmented Signal (max noise amplitude: {max_noise_amplitude})')

> Original Signal


> Augmented Signal (max noise amplitude: 0.001)


> Augmented Signal (max noise amplitude: 0.005)


> Augmented Signal (max noise amplitude: 0.01)


> Augmented Signal (max noise amplitude: 0.05)


### Time Stretching

In [31]:
play_audio(y=y, sr=sr, title='Original Signal')
for stretch_rate in [0.2, 0.5, 0.8, 1.0, 1.2, 1.5, 2.0]:
    augmented_y, params_str = augment_audio(y=y, sample_rate=sr, apply_time_stretch=True, min_stretch_rate=stretch_rate,max_stretch_rate=stretch_rate)
    play_audio(y=augmented_y, sr=sr, title=f'Augmented Signal (stretch rate: {stretch_rate})')

> Original Signal


> Augmented Signal (stretch rate: 0.2)


> Augmented Signal (stretch rate: 0.5)


> Augmented Signal (stretch rate: 0.8)


> Augmented Signal (stretch rate: 1.0)


> Augmented Signal (stretch rate: 1.2)


> Augmented Signal (stretch rate: 1.5)


> Augmented Signal (stretch rate: 2.0)


### Pitch Scaling

In [33]:
play_audio(y=y, sr=sr, title='Original Signal')
for semitones in [-6, -3,-1,1, 3, 6]:
    augmented_y, params_str = augment_audio(y=y, sample_rate=sr, apply_pitch_scaling=True, min_semitones=semitones, max_semitones=semitones)
    play_audio(y=augmented_y, sr=sr, title=f'Augmented Signal (semitones: {semitones})')

> Original Signal


> Augmented Signal (semitones: -6)


> Augmented Signal (semitones: -3)


> Augmented Signal (semitones: -1)


> Augmented Signal (semitones: 1)


> Augmented Signal (semitones: 3)


> Augmented Signal (semitones: 6)


### Polarity Inversion

In [11]:
play_audio(y=y, sr=sr, title='Original Signal')
augmented_y, params_str = augment_audio(y=y, sample_rate=sr, apply_polarity_inversion=True)
play_audio(y=augmented_y, sr=sr, title=f'Augmented Signal (polarity inversion)')

> Original Signal


> Augmented Signal (polarity inversion)


### Random Gain

In [12]:
play_audio(y=y, sr=sr, title='Original Signal')
for gain_db in [-50, -10, -5, 5, 10, 20]:
    augmented_y, params_str = augment_audio(y=y, sample_rate=sr, apply_random_gain=True, min_gain_db=gain_db,
                                            max_gain_db=gain_db)
    play_audio(y=augmented_y, sr=sr, title=f'Augmented Signal (gain: {gain_db} dB)')

> Original Signal


> Augmented Signal (gain: -50 dB)


> Augmented Signal (gain: -10 dB)


> Augmented Signal (gain: -5 dB)


> Augmented Signal (gain: 5 dB)


> Augmented Signal (gain: 10 dB)


> Augmented Signal (gain: 20 dB)


### Time Shifting

Décaler légèrement l'audio dans le temps, soit en le déplaçant vers la gauche (en avance), soit vers la droite (en retard). Cela peut aider le modèle à être plus robuste aux petits décalages temporels.

In [13]:
play_audio(y=y, sr=sr, title="Original Signal")

for max_shift_fraction in [0.05, 0.1, 0.2, 0.8]:
    augmented_y, params_str = augment_audio(y=y, sample_rate=sr, apply_time_shift=True,m
    play_audio(y=augmented_y, sr=sr, title=f'Augmented Signal (max_shift_samples: {max_shift_fraction})')


> Original Signal


> Augmented Signal (max_shift_samples: 0.05)


> Augmented Signal (max_shift_samples: 0.1)


> Augmented Signal (max_shift_samples: 0.2)


> Augmented Signal (max_shift_samples: 0.8)


### Impulse Response

In [14]:
# acouctic impulse responses
air_files = get_all_audios_files(AIRS_PATH)
air_files

['G:\\Shared drives\\PFE - ING3 Mlamali\\DrumClassifier - Acoustic Impulse Responses\\1st_baptist_nashville_balcony_resampled.wav',
 'G:\\Shared drives\\PFE - ING3 Mlamali\\DrumClassifier - Acoustic Impulse Responses\\1st_baptist_nashville_far_close_resampled.wav',
 'G:\\Shared drives\\PFE - ING3 Mlamali\\DrumClassifier - Acoustic Impulse Responses\\1st_baptist_nashville_far_wide_resampled.wav']

In [15]:
play_audio(y=y, sr=sr, title="Original Signal")
# get random index on airs_files
idx_air = np.random.randint(len(air_files))

for air_file in air_files:
    augmented_y, params_str = augment_audio(y=y, sample_rate=sr, apply_impulse_response=True, ir_path=air_file)
    play_audio(y=augmented_y, sr=sr, title=f'Augmented Signal (impulse : {os.path.basename(air_file)})')

# en mettant tous les fichiers, il va choisir un aux hasard
augmented_y, params_str = augment_audio(y=y, sample_rate=sr, apply_impulse_response=True, ir_path=air_files)
play_audio(y=augmented_y, sr=sr, title=f'Augmented Signal (impulses)')


> Original Signal


> Augmented Signal (impulse : 1st_baptist_nashville_balcony_resampled.wav)


> Augmented Signal (impulse : 1st_baptist_nashville_far_close_resampled.wav)


> Augmented Signal (impulse : 1st_baptist_nashville_far_wide_resampled.wav)


> Augmented Signal (impulses)


### Filtering

1. Sons aigus (cymbales, hi-hats) :
- LowPassFilter : Utilisez un filtre passe-bas avec une fréquence de coupure maximale relativement élevée (par exemple, 12 000 Hz ou plus) pour conserver l'énergie dans les hautes fréquences.
- HighPassFilter : Appliquez un filtre passe-haut avec une fréquence de coupure minimale plus élevée (par exemple, 500 Hz) pour éliminer les basses fréquences indésirables.
- BandPassFilter : Sélectionnez une plage de fréquences qui se concentre sur les hautes fréquences, comme min_cutoff_freq = 500 Hz et max_cutoff_freq = 12 000 Hz ou plus.
- BandStopFilter : Employez un filtre coupe-bande pour atténuer certaines fréquences spécifiques qui pourraient être problématiques dans les sons aigus, comme min_cutoff_freq = 1000 Hz et max_cutoff_freq = 3000 Hz.

2. Sons moyennement graves (claps, snares) :
- LowPassFilter : Privilégiez un filtre passe-bas avec une fréquence de coupure maximale modérée (par exemple, 8000 Hz) pour atténuer les hautes fréquences inutiles tout en préservant les fréquences caractéristiques de ces sons.
- HighPassFilter : Optez pour un filtre passe-haut avec une fréquence de coupure minimale modérée (par exemple, 200 Hz) pour éliminer les basses fréquences indésirables.
- BandPassFilter : Choisissez une plage de fréquences qui met l'accent sur les fréquences moyennes, par exemple min_cutoff_freq = 200 Hz et max_cutoff_freq = 8000 Hz.
- BandStopFilter : Utilisez un filtre coupe-bande pour atténuer certaines fréquences spécifiques qui pourraient être problématiques dans les sons moyennement graves, comme min_cutoff_freq = 500 Hz et max_cutoff_freq = 2000 Hz.

3. Sons graves (808, kick, tom) :
- LowPassFilter : Adoptez un filtre passe-bas avec une fréquence de coupure maximale plus basse (par exemple, 5000 Hz) pour atténuer les hautes fréquences inutiles.
- HighPassFilter : Appliquez un filtre passe-haut avec une fréquence de coupure minimale plus basse (par exemple, 40 Hz) pour conserver les basses fréquences essentielles.
- BandPassFilter : Sélectionnez une plage de fréquences qui se concentre sur les basses fréquences, comme min_cutoff_freq = 40 Hz et max_cutoff_freq = 5000 Hz.
- BandStopFilter : Employez un filtre coupe-bande pour atténuer certaines fréquences spécifiques qui pourraient être problématiques dans les sons graves, comme min_cutoff_freq = 100 Hz et max_cutoff_freq = 500 Hz.


4. Conseils généraux :
- Variez les fréquences de coupure et les plages de fréquences pour créer des variations subtiles dans les sons augmentés.
- Écoutez les résultats pour vous assurer que les filtres ne dégradent pas la qualité des sons de manière indésirable.
- Expérimentez avec différents types de filtres (passe-bas, passe-haut, passe-bande, coupe-bande) pour créer une variété de sons augmentés.


#### Low Pass

- min_cutoff_freq : Choisissez une valeur qui permet de conserver les fréquences basses importantes pour les sons de batterie (par exemple, grosse caisse et caisse claire). Une valeur de 150 Hz peut être un bon point de départ.
- max_cutoff_freq : Choisissez une valeur qui supprime les fréquences aiguës inutiles ou indésirables pour la classification. Une valeur de 7500 Hz peut être un bon point de départ.



In [16]:
play_audio(y=y, sr=sr, title='Original Signal')

# low pass
augmented_y, params_str = augment_audio(y=y, sample_rate=sr, apply_filtering=True, filter_type="low_pass", min_filter_freq=150, max_filter_freq=7500)
play_audio(y=augmented_y, sr=sr, title=f'Augmented Signal (low pass)')

> Original Signal


> Augmented Signal (low pass)


#### High Pass

- min_cutoff_freq : Choisissez une valeur qui supprime les fréquences basses indésirables pour la classification. Une valeur de 40 Hz peut être un bon point de départ.
- max_cutoff_freq : Choisissez une valeur qui permet de conserver les fréquences aiguës importantes pour les sons de batterie (par exemple, cymbales). Une valeur de 1000 Hz peut être un bon point de départ.


In [17]:
play_audio(y=y, sr=sr, title='Original Signal')

# low pass
augmented_y, params_str = augment_audio(y=y, sample_rate=sr, apply_filtering=True, filter_type="high_pass", min_filter_freq=40, max_filter_freq=1000)
play_audio(y=augmented_y, sr=sr, title=f'Augmented Signal (high pass)')

> Original Signal


> Augmented Signal (high pass)


#### Band Pass

In [18]:
play_audio(y=y, sr=sr, title='Original Signal')

# low pass
augmented_y, params_str = augment_audio(y=y, sample_rate=sr, apply_filtering=True, filter_type="band_pass")
play_audio(y=augmented_y, sr=sr, title=f'Augmented Signal (band pass)')

> Original Signal


> Augmented Signal (band pass)


#### Band Reject

In [19]:
play_audio(y=y, sr=sr, title='Original Signal')

# low pass
augmented_y, params_str = augment_audio(y=y, sample_rate=sr, apply_filtering=True, filter_type="band_reject")
play_audio(y=augmented_y, sr=sr, title=f'Augmented Signal (band pass)')

> Original Signal


> Augmented Signal (band pass)


## Random Augmentation

In [1]:
play_audio(y=y, sr=sr, title='Original Signal')
augmented_y, params_str = augment_audio(y=y, sample_rate=sr,
                                        apply_noise=True,p_noise=0.5,max_noise_amplitude=random.choice([0.001,0.005]),
                                        apply_time_stretch=True, p_time_stretch=0.8, min_stretch_rate=0.6, max_stretch_rate=1.4,
                                        apply_pitch_scaling=True, p_pitch_scaling=0.5, min_semitones=-4, max_semitones=4,
                                        apply_filtering=True,p_filtering=0.1, filter_type=random.choice(["low_pass","high_pass","band_pass","band_reject"]), min_filter_freq=40, max_filter_freq=1000,
                                        apply_impulse_response=True, p_impulse_response=0.1,
                                        apply_time_shift=True, p_time_shift=0.5, max_shift_second=0.2,
                                        apply_random_gain=True,

                                        apply_polarity_inversion=True)


play_audio(y=augmented_y, sr=sr, title=f'Augmented Signal')

NameError: name 'np' is not defined

## Create augmented Dataset